# **Nonlinear Optimization**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1CtEj1XBifRGZFkaZxdI_0vEWYWfHiLAS?usp=sharing)

Nonlinear optimization involves finding the best solution to a mathematical programming problem in which the objective function and constraints are not necessarily linear. Because nonlinear models include literally all kinds of models *except* linear ones, it is not surprising that this category is a very broad one, and nonlinear optimization must incorporate a wide variety of approaches to solving problems.<br >
The world is full of systems that do not behave linearly. For example, allowing a tree to grow twice as long does not necessarily double the resulting timber harvest; and tripling the amount of fertilizer applied to a wheat field does not necessarily triple the yield (and might even kill the crop!). In a distributed computing system networked for interprocessor communication, doubling the speed of the processors does not mean that all distributed computations will be completed in half the time, because interactions among processors now could foil the anticipated speedup in throughput.<br >
This chapter examines optimization from a very general point of view. We will consider both unconstrained and constrained models. **Unconstrained optimization** is often dealt with through the use of differential calculus to determine maximum or minimum points of an objective function. Constrained models may present us with systems of equations to be solved. In either case, the classical underlying theories that describe the characteristics of an optimum do not necessarily provide the practical *methods* that are suitable for efficient numerical computation of the desired solutions. Nevertheless, a thorough grasp of the subject of nonlinear optimization requires an understanding of both the mathematical foundations of optimization as well as the algorithms that have been developed for obtaining solutions. This chapter is intended to provide insights from both of these perspectives. We will first look at an example of a nonlinear programming problem formulation.<br >

<ul />

**Example 5.1**

Suppose we want to determine a production schedule over several time periods, where the demand in each period can be met with either products in inventory at the end of the previous period or production during the current period. Let the $T$ time periods be indexed by $i = 1, 2, …, T,$ and let $D_i$ be the known demand at time period $i$. Equipment capacities and material limitations restrict production to at most $E_i$ units during period $i$. The labor force $L_i$ during period $i$ can be adjusted according to demand, but hiring and firing is costly, so a cost $C_L$ is applied to the square of the net change in labor force size from one period to the next. The productivity (number of units produced) of each worker during any period $i$ is given as $P_i$
. The number of units of inventory at the end of period $i$ is $I_i$
, and the cost of carrying a unit of inventory into the next period is $C_I$
. The production scheduling problem is then to determine feasible labor force and inventory 
levels in order to meet demand at minimum total cost. The decision variables are the $L_i$
and $I_i$ for $i = 1, …, T$. The initial labor force and inventory levels are given as $L_0$ and $I_0$, 
respectively. Therefore, we wish to

$$
\begin{aligned}
& \text{minimize} && \sum_{i=1}^{T} C_L (L_i - L_{i-1})^2 + C_I I_i \\
& \text{subject to} && L_i \cdot P_i \le E_i && \text{equipment capacities for } i = 1, \dots, T \\
& && I_{i-1} + L_i \cdot P_i \ge D_i && \text{demand for } i=1, \dots, T \\
& && I_i = I_{i-1} + L_i \cdot P_i - D_i && \text{inventory for } i=1, \dots, T \\
& && L_i, I_i \ge 0 && \text{for } i=1, \dots, T
\end{aligned}
$$

This nonlinear model has a quadratic objective function, but linear constraints, and it happens to involve discrete decision variables. Other nonlinear models may involve continuous processes that are represented by time-integrated functions or flow problems described by differential equations.

</ul>

In [ ]:
import numpy as np
from scipy.optimize import minimize, Bounds, LinearConstraint

class ProductionScheduler:
    def __init__(self, T, CL, CI, E, D, P, L0, I0):
        self.T = T
        self.CL = CL
        self.CI = CI
        self.E = E
        self.D = D
        self.P = P
        self.L0 = L0
        self.I0 = I0
        self.num_vars = 2 * T

    def _objective_function(self):
        def objective(x):
            L = x[:self.T]
            I = x[self.T:]

            # Labor cost term: sum(CL * (L_i - L_{i-1})^2)
            L_prev = np.concatenate(([self.L0], L[:-1]))
            labor_cost = np.sum(self.CL * (L - L_prev)**2)

            # Inventory cost term: sum(CI * I_i)
            inventory_cost = np.sum(self.CI * I)

            return labor_cost + inventory_cost
        return objective

    def _get_constraints(self):
        # Constraint 1: Equipment capacities -> L_i * P_i <= E_i
        A_capacity = np.zeros((self.T, self.num_vars))
        b_capacity = np.array(self.E)
        for i in range(self.T):
            A_capacity[i, i] = self.P[i]
        capacity_constraint = LinearConstraint(A_capacity, -np.inf, b_capacity)

        # Constraint 2: Demand satisfaction -> I_{i-1} + L_i * P_i >= D_i
        A_demand = np.zeros((self.T, self.num_vars))
        b_demand = -np.array(self.D)
        for i in range(self.T):
            A_demand[i, i] = -self.P[i]
            if i > 0:
                A_demand[i, self.T + i - 1] = -1
            else:
                b_demand[i] -= -self.I0
        demand_constraint = LinearConstraint(A_demand, -np.inf, b_demand)

        # Constraint 3: Inventory balance -> I_i = I_{i-1} + L_i * P_i - D_i
        A_balance = np.zeros((self.T, self.num_vars))
        b_balance = -np.array(self.D)
        for i in range(self.T):
            A_balance[i, i] = -self.P[i]
            A_balance[i, self.T + i] = 1
            if i > 0:
                A_balance[i, self.T + i - 1] = -1
            else:
                b_balance[i] -= -self.I0
        balance_constraint = LinearConstraint(A_balance, b_balance, b_balance)

        return [capacity_constraint, demand_constraint, balance_constraint]

    def solve(self):
        objective = self._objective_function()
        constraints = self._get_constraints()

        # Bounds for all variables (L_i >= 0, I_i >= 0)
        bounds = Bounds(np.zeros(self.num_vars), np.inf * np.ones(self.num_vars))

        # Initial guess
        x0 = np.ones(self.num_vars)

        # Run the optimization
        result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)

        # Print the results
        if result.success:
            print("Optimization successful!")
            L_optimized = result.x[:self.T]
            I_optimized = result.x[self.T:]
            print("\nOptimal Labor Force (L):")
            print(f"L0: {self.L0:.2f}")
            for i in range(self.T):
                print(f"L{i+1}: {L_optimized[i]:.2f}")

            print("\nOptimal Inventory (I):")
            print(f"I0: {self.I0:.2f}")
            for i in range(self.T):
                print(f"I{i+1}: {I_optimized[i]:.2f}")

            print(f"\nTotal Minimum Cost: {result.fun:.2f}")
        else:
            print("Optimization failed.")
            print(f"Reason: {result.message}")
        
        return result

if __name__ == '__main__':
    T_periods = 3
    cost_labor_change = 50.0
    cost_inventory_unit = 10.0
    equipment_capacity = [200, 250, 220]
    demand = [150, 180, 200]
    production_rate = [1.0, 1.0, 1.0]
    initial_labor = 100.0
    initial_inventory = 50.0

    # Create an instance of the scheduler and solve the problem
    scheduler = ProductionScheduler(T_periods, cost_labor_change, cost_inventory_unit,
                                   equipment_capacity, demand, production_rate,
                                   initial_labor, initial_inventory)
    scheduler.solve()

Optimization successful!

Optimal Labor Force (L):
L0: 100.00
L1: 138.51
L2: 164.28
L3: 177.21

Optimal Inventory (I):
I0: 50.00
I1: 38.51
I2: 22.79
I3: 0.00

Total Minimum Cost: 116327.89
